# **PROYEK PRA UTS PROYEK SAIN DATA**
## **DOKUMENTASI PROSES PEMBUATAN WORKFLOW**

## **Tahap Koneksi ke Database**



Langkah pertama adalah menghubungkan KNIME Analytics Platform dengan database PostgreSQL yang berisi data penelitian.
Untuk melakukan koneksi digunakan node PostgreSQL Connector, yang berfungsi sebagai jembatan antara KNIME dan database.

Konfigurasi koneksi:

- Host: localhost

- Database: db_ecoli

- Port: 2005

- Username: postgres

- Password: muniiff26

Node ini memastikan bahwa KNIME dapat mengakses data yang tersimpan di dalam database PostgreSQL secara langsung tanpa perlu ekspor manual.
Apabila konfigurasi berhasil, maka akan muncul status “connected” pada node tersebut.

## **Pemilihan Tabel Database**



Setelah koneksi berhasil, tahap berikutnya adalah menentukan tabel yang akan digunakan untuk analisis.
Hal ini dilakukan menggunakan node DB Table Selector, yang berfungsi memilih tabel tertentu dari database yang sudah terhubung.

Pengaturan:

- Schema: public

- Table: ecoli_data

Dengan konfigurasi tersebut, node ini mengambil tabel ecoli_data dari schema publik database PostgreSQL.
Tabel inilah yang akan menjadi sumber utama data dalam proses analisis berikutnya.

## **Membaca Data dari Database**


Node ini berperan untuk mengekstrak data dari database ke dalam workflow KNIME.
Data yang sebelumnya hanya berada di sistem database kini ditransformasikan ke dalam bentuk tabel internal KNIME sehingga dapat diolah menggunakan node analisis lainnya.

Tahap ini juga membantu dalam memastikan bahwa struktur data (kolom, tipe data, dan isi) sudah terbaca dengan benar oleh sistem KNIME sebelum melanjutkan ke tahapan pra-pemrosesan.

## **Pemeriksaan dan Penyaringan Missing Value**



Pada tahap ini digunakan node Missing Value Filter yang berfungsi untuk mendeteksi keberadaan nilai kosong (missing value) pada dataset.
Langkah ini penting untuk menjaga kualitas data karena keberadaan nilai kosong dapat mempengaruhi hasil analisis atau model.

Setelah dilakukan proses pemeriksaan terhadap data ecoli_data, diperoleh hasil bahwa seluruh atribut memiliki nilai lengkap dan tidak ditemukan adanya missing value.
Oleh karena itu, proses pembersihan data pada tahap ini tidak diperlukan dan dapat langsung dilanjutkan ke tahap berikutnya.

## **Deteksi Data Outlier Menggunakan Metode ABOD**




Setelah memastikan data bersih, langkah berikutnya adalah mendeteksi keberadaan outlier (data pencilan) menggunakan metode Angle-Based Outlier Detection (ABOD).
Tahapan ini dilakukan melalui node Python Script, yang memanfaatkan pustaka PyCaret untuk analisis anomali.

**Penjelasan:**

setup() digunakan untuk menginisialisasi lingkungan kerja PyCaret dengan data numerik yang relevan.

create_model("abod", fraction=0.05) membuat model deteksi outlier menggunakan algoritma ABOD, di mana sekitar 5% data dianggap sebagai pencilan.

assign_model() menambahkan label hasil deteksi pada data, dengan nilai:

0 → data normal

1 → data outlier

Proses merge dilakukan agar hasil deteksi digabungkan kembali dengan kolom non-numerik seperti id, name, dan class.

Hasil akhirnya berupa tabel yang telah memiliki kolom tambahan “Anomaly” sebagai indikator data normal atau pencilan.

## **Visualisasi Hasil Deteksi Outlier**




Tahapan ini bertujuan untuk memvisualisasikan posisi data outlier dan data normal agar lebih mudah dianalisis.
Visualisasi dilakukan menggunakan metode Principal Component Analysis (PCA) untuk mereduksi dimensi data numerik menjadi dua komponen utama.

**Penjelasan:**

- PCA menurunkan dimensi data menjadi dua sumbu utama agar data bisa divisualisasikan dalam bentuk 2D.

- Hasil reduksi digabung dengan kolom Anomaly agar tiap titik dapat dibedakan antara data normal (0) dan outlier (1).

- Plot menampilkan dua kelompok data:

- Titik berlabel 0 menunjukkan data normal.

- Titik berlabel 1 menunjukkan data yang terdeteksi sebagai outlier.

- Visualisasi ini memudahkan untuk melihat distribusi dan perbedaan posisi antara data normal dan anomali.

## **Tahap Pembersihan Outlier**

Setelah diketahui data mana yang termasuk outlier, tahap berikutnya adalah menghapus atau menyaring data pencilan dari dataset.
Namun, dalam kasus ini beberapa kelas seperti imL dan imS tetap dipertahankan meskipun mungkin termasuk kategori outlier karena keduanya dianggap penting secara domain data.

**Penjelasan:**

Data disaring agar hanya menyisakan:

Data normal (Anomaly = 0),

dan dua kelas yang dianggap relevan (imL dan imS).

Baris data selain kondisi tersebut dihapus dari dataset.

Hasil akhirnya adalah data bersih yang siap digunakan untuk proses penyeimbangan (balancing).

## **Proses Penyeimbangan Data (Data Balancing)**

Dataset hasil pembersihan memiliki jumlah data tiap kelas yang tidak seimbang.
Oleh karena itu, dilakukan proses penyeimbangan kelas menggunakan metode SMOTE (Synthetic Minority Oversampling Technique) yang mampu membuat data sintetis untuk kelas dengan jumlah data sedikit.

**Penjelasan:**

SMOTE menambah data secara sintetis untuk kelas minoritas dengan memanfaatkan teknik interpolasi antar tetangga terdekat.

Parameter k_neighbors=1 digunakan karena distribusi data per kelas cukup kecil.

Setelah resampling, data numerik dan label kelas digabung kembali menjadi satu tabel (merge_smote).

Hasil akhirnya adalah dataset yang sudah memiliki distribusi kelas yang seimbang, siap digunakan untuk modeling.

## **Distribusi Data Setelah Penyeimbangan**

Tahapan terakhir adalah memeriksa kembali distribusi jumlah data setiap kelas setelah dilakukan SMOTE.
Visualisasi distribusi ini menunjukkan bahwa seluruh kelas kini memiliki proporsi data yang hampir sama.

Hal ini menandakan bahwa proses balancing berhasil dan dataset telah siap untuk digunakan pada tahap pemodelan (model training) atau analisis lanjutan.

## **Kesimpulan Akhir**

Proses pengerjaan workflow proyek Pra UTS Penambangan Sains Data pada KNIME mencakup seluruh langkah penting dalam tahap preprocessing data, meliputi:

Menghubungkan KNIME dengan database PostgreSQL.

Mengambil tabel ecoli_data sebagai sumber dataset.

Membaca data ke dalam workflow KNIME.

Memastikan tidak ada missing value.

Mendeteksi outlier menggunakan metode ABOD.

Melakukan visualisasi distribusi outlier.

Membersihkan data pencilan yang tidak relevan.

Melakukan balancing data menggunakan metode SMOTE.

Hasil akhir dari seluruh proses ini adalah dataset bersih, bebas outlier, dan seimbang, yang siap digunakan untuk tahapan analisis atau pembangunan model klasifikasi dan clustering selanjutnya.